In [204]:
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
import pyodbc
import pandas_access as pa
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, roc_auc_score,classification_report
from sklearn.model_selection import cross_val_predict,GridSearchCV, StratifiedKFold
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from sklearn.covariance import EllipticEnvelope
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler,MinMaxScaler  
from sklearn.neural_network import MLPClassifier
from tensorflow import keras
import tensorflow as tf
import imblearn
from imblearn.over_sampling import SMOTE,BorderlineSMOTE,ADASYN,SVMSMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline as imbpipeline
import seaborn as sns
from collections import Counter
import loras

In [97]:
pd.set_option('display.max_columns',200)
pd.set_option('display.max_rows',200)

In [238]:
dataset_close=pd.read_csv('data\hospital_closure\hospital_closure.csv')

In [99]:
list(cur_2017.tables())

[('data\\hospital_compare\\hc_apr2017\\Hospital.mdb', None, 'MSysAccessObjects', 'SYSTEM TABLE', None),
 ('data\\hospital_compare\\hc_apr2017\\Hospital.mdb', None, 'MSysAccessXML', 'SYSTEM TABLE', None),
 ('data\\hospital_compare\\hc_apr2017\\Hospital.mdb', None, 'MSysACEs', 'SYSTEM TABLE', None),
 ('data\\hospital_compare\\hc_apr2017\\Hospital.mdb', None, 'MSysIMEXColumns', 'SYSTEM TABLE', None),
 ('data\\hospital_compare\\hc_apr2017\\Hospital.mdb', None, 'MSysIMEXSpecs', 'SYSTEM TABLE', None),
 ('data\\hospital_compare\\hc_apr2017\\Hospital.mdb', None, 'MSysNameMap', 'SYSTEM TABLE', None),
 ('data\\hospital_compare\\hc_apr2017\\Hospital.mdb', None, 'MSysNavPaneGroupCategories', 'SYSTEM TABLE', None),
 ('data\\hospital_compare\\hc_apr2017\\Hospital.mdb', None, 'MSysNavPaneGroups', 'SYSTEM TABLE', None),
 ('data\\hospital_compare\\hc_apr2017\\Hospital.mdb', None, 'MSysNavPaneGroupToObjects', 'SYSTEM TABLE', None),
 ('data\\hospital_compare\\hc_apr2017\\Hospital.mdb', None, 'MSysNavPane

In [239]:
MDB_2017='data\hospital_compare\hc_apr2017\Hospital.mdb'
MDB_2016 = 'data\hospital_compare\hc_may2016\Hospital.mdb'
MDB_2015='data\hospital_compare\Copy of HospitalMay2015.mdb'
MDB_2014='data\hospital_compare\HC_May2014.mdb'

DRV = '{Microsoft Access Driver (*.mdb, *.accdb)}'
PWD = 'pw'

con_2017 = pyodbc.connect('DRIVER={};DBQ={};PWD={}'.format(DRV,MDB_2017,PWD))
con_2016 = pyodbc.connect('DRIVER={};DBQ={};PWD={}'.format(DRV,MDB_2016,PWD))
con_2015 = pyodbc.connect('DRIVER={};DBQ={};PWD={}'.format(DRV,MDB_2015,PWD))
con_2014 = pyodbc.connect('DRIVER={};DBQ={};PWD={}'.format(DRV,MDB_2014,PWD))
cur_2017 = con_2017.cursor()
cur_2016 = con_2016.cursor()
cur_2015 = con_2015.cursor()
cur_2014 = con_2014.cursor()

In [240]:
table_name = '[Medicare Hospital Spending by Claim]'

query = "SELECT * FROM {}".format(table_name)

query3 = "SELECT * FROM {}.COLUMNS".format(table_name)
rows_2017 = cur_2017.execute(query).fetchall()
rows_2016 = cur_2016.execute(query).fetchall()
rows_2015 = cur_2015.execute(query).fetchall()
rows_2014 = cur_2014.execute(query).fetchall()

In [ ]:
cur_2016.execute(query).fetchone()

('SOUTHEAST ALABAMA MEDICAL CENTER', '010001', 'AL', '1 to 3 days Prior to Index Hospital Admission', 'Home Health Agency', '$12', '$14', '$13', '0.06%', '0.07%', '0.07%', '01/01/2014', '12/31/2014')

In [ ]:
for row in cur_2017.columns(table='Medicare Hospital Spending by Claim'):
    print(row.column_name)

Hospital_Name
Provider_ID
State
Period
Claim_Type
Avg_Spending_Per_Episode_Hospital
Avg_Spending_Per_Episode_State
Avg_Spending_Per_Episode_Nation
Percent_of_Spending_Hospital
Percent_of_Spending_State
Percent_of_Spending_Nation
Start_Date
End_Date


In [241]:
arr_2017=np.array(rows_2017)
dataset_2017=pd.DataFrame(arr_2017[:,[1,5,6,7,8,9,10]],columns=['id','1','2','3','4','5','6'])
arr_2016=np.array(rows_2016)
dataset_2016=pd.DataFrame(arr_2016[:,[1,5,6,7,8,9,10]],columns=['id','1','2','3','4','5','6'])
arr_2015=np.array(rows_2015)
dataset_2015=pd.DataFrame(arr_2015[:,[1,5,6,7,8,9,10]],columns=['id','1','2','3','4','5','6'])
arr_2014=np.array(rows_2014)
dataset_2014=pd.DataFrame(arr_2014[:,[1,5,6,7,8,9,10]],columns=['id','1','2','3','4','5','6'])

In [105]:
dataset_2014

,id,1,2,3,4,5,6
0,010001,$11,$14,$13,0.06%,0.08%,0.07%
1,010001,$4,$2,$1,0.02%,0.01%,0.00%
2,010001,$9,$5,$5,0.05%,0.03%,0.03%
3,010001,$71,$51,$63,0.38%,0.29%,0.34%
4,010001,$1,$2,$2,0.01%,0.01%,0.01%
...,...,...,...,...,...,...,...
143435,670082,$537,$647,$624,2.92%,3.25%,3.33%
143436,670082,"$2,682","$2,516","$2,924",14.56%,12.65%,15.63%
143437,670082,$135,$132,$112,0.73%,0.66%,0.60%
143438,670082,"$1,228","$1,124","$1,005",6.67%,5.65%,5.37%


In [242]:
dataset_2017['1']=dataset_2017['1'].str.strip("$")
dataset_2017['1']=dataset_2017['1'].str.replace(',', '')
dataset_2017['2']=dataset_2017['2'].str.strip("$")
dataset_2017['2']=dataset_2017['2'].str.replace(',', '')
dataset_2017['3']=dataset_2017['3'].str.strip("$")
dataset_2017['3']=dataset_2017['3'].str.replace(',', '')
dataset_2016['1']=dataset_2016['1'].str.strip("$")
dataset_2016['1']=dataset_2016['1'].str.replace(',', '')
dataset_2016['2']=dataset_2016['2'].str.strip("$")
dataset_2016['2']=dataset_2016['2'].str.replace(',', '')
dataset_2016['3']=dataset_2016['3'].str.strip("$")
dataset_2016['3']=dataset_2016['3'].str.replace(',', '')
dataset_2015['1']=dataset_2015['1'].str.strip("$")
dataset_2015['1']=dataset_2015['1'].str.replace(',', '')
dataset_2015['2']=dataset_2015['2'].str.strip("$")
dataset_2015['2']=dataset_2015['2'].str.replace(',', '')
dataset_2015['3']=dataset_2015['3'].str.strip("$")
dataset_2015['3']=dataset_2015['3'].str.replace(',', '')
dataset_2014['1']=dataset_2014['1'].str.strip("$")
dataset_2014['1']=dataset_2014['1'].str.replace(',', '')
dataset_2014['2']=dataset_2014['2'].str.strip("$")
dataset_2014['2']=dataset_2014['2'].str.replace(',', '')
dataset_2014['3']=dataset_2014['3'].str.strip("$")
dataset_2014['3']=dataset_2014['3'].str.replace(',', '')

In [243]:
dataset_2017['4']=dataset_2017['4'].str.strip("%").astype(float)/100
dataset_2017['5']=dataset_2017['5'].str.strip("%").astype(float)/100
dataset_2017['6']=dataset_2017['6'].str.strip("%").astype(float)/100
dataset_2016['4']=dataset_2016['4'].str.strip("%").astype(float)/100
dataset_2016['5']=dataset_2016['5'].str.strip("%").astype(float)/100
dataset_2016['6']=dataset_2016['6'].str.strip("%").astype(float)/100
dataset_2015['4']=dataset_2015['4'].str.strip("%").astype(float)/100
dataset_2015['5']=dataset_2015['5'].str.strip("%").astype(float)/100
dataset_2015['6']=dataset_2015['6'].str.strip("%").astype(float)/100
dataset_2014['4']=dataset_2014['4'].str.strip("%").astype(float)/100
dataset_2014['5']=dataset_2014['5'].str.strip("%").astype(float)/100
dataset_2014['6']=dataset_2014['6'].str.strip("%").astype(float)/100

In [244]:
for col in dataset_2017:
    dataset_2017[col]=pd.to_numeric(dataset_2017[col])
for col in dataset_2016:
    dataset_2016[col]=pd.to_numeric(dataset_2016[col])
for col in dataset_2015:
    dataset_2015[col]=pd.to_numeric(dataset_2015[col])
for col in dataset_2014:
    dataset_2014[col]=pd.to_numeric(dataset_2014[col])

In [245]:
close_2016=dataset_close[dataset_close['year_close_combined']==2016]
close_2015=dataset_close[dataset_close['year_close_combined']==2015]
close_2014=dataset_close[dataset_close['year_close_combined']==2014]
close_2013=dataset_close[dataset_close['year_close_combined']==2013]
dataset_2017['Closure']=dataset_2017.id.apply(lambda x:1 if x in np.array(close_2016['id']) else 0)
dataset_2016['Closure']=dataset_2016.id.apply(lambda x:1 if x in np.array(close_2015['id']) else 0)
dataset_2015['Closure']=dataset_2015.id.apply(lambda x:1 if x in np.array(close_2014['id']) else 0)
dataset_2014['Closure']=dataset_2014.id.apply(lambda x:1 if x in np.array(close_2013['id']) else 0)

In [253]:
dataset_2014[dataset_2014['Closure']==1]['id'].value_counts().sort_index()

10066     44
30126     44
50349     44
50768     44
140151    44
360113    44
450283    44
450770    44
450813    44
450839    44
490012    44
Name: id, dtype: int64

In [252]:
dataset_2016.loc[dataset_2016['id']==200025,'Closure']=0
dataset_2014.loc[dataset_2014['id']==180149,'Closure']=0
dataset_2014.loc[dataset_2014['id']==450832,'Closure']=0
dataset_2014.loc[dataset_2014['id']==670052,'Closure']=0

In [254]:
dataset_2014['Closure'].value_counts(dropna=False)

0    142956
1       484
Name: Closure, dtype: int64

In [255]:
dataset_2017=dataset_2017.to_numpy().reshape(int(dataset_2017.shape[0]/22),8*22)
dataset_2016=dataset_2016.to_numpy().reshape(int(dataset_2016.shape[0]/22),8*22)
dataset_2015=dataset_2015.to_numpy().reshape(int(dataset_2015.shape[0]/22),8*22)
dataset_2014=dataset_2014.to_numpy().reshape(int(dataset_2014.shape[0]/22),8*22)


In [256]:
arr1=[i*8 for i in range(22)]
arr2=[i*8+7 for i in range(22)]
arr=np.array([arr1,arr2]).flatten()
arr.sort()

In [257]:
dataset_2017=np.delete(dataset_2017,arr[:-1],axis=1)
dataset_2016=np.delete(dataset_2016,arr[:-1],axis=1)
dataset_2015=np.delete(dataset_2015,arr[:-1],axis=1)
dataset_2014=np.delete(dataset_2014,arr[:-1],axis=1)

In [258]:
dataset_2014.shape

(6520, 133)

In [259]:
dataset_=np.concatenate((dataset_2014,dataset_2015,dataset_2016,dataset_2017),axis=0)

In [260]:
dataset_.shape

(16124, 133)

In [261]:
train, test = train_test_split(dataset_, test_size=0.3)

In [262]:
X_train = train[:,:-1]
Y_train = train[:,-1].astype(int)
X_test  = test[:,:-1]
Y_test  = test[:,-1].astype(int)
X_train.shape, Y_train.shape,X_test.shape, Y_test.shape

((11286, 132), (11286,), (4838, 132), (4838,))

In [263]:
m=RandomForestClassifier()
m.fit(X_train, Y_train)
predict_y = m.predict(X_test)
print('ROCAUC score:',roc_auc_score(Y_test, predict_y))
print('Accuracy score:',accuracy_score(Y_test, predict_y))
print('F1 score:',f1_score(Y_test, predict_y))
print(classification_report(Y_test, predict_y))
confusion_matrix(Y_test, predict_y)

ROCAUC score: 0.71875
Accuracy score: 0.9981397271599834
F1 score: 0.6086956521739131
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      4822
           1       1.00      0.44      0.61        16

    accuracy                           1.00      4838
   macro avg       1.00      0.72      0.80      4838
weighted avg       1.00      1.00      1.00      4838



array([[4822,    0],
       [   9,    7]], dtype=int64)

In [264]:


smote = ADASYN()

# fit target and predictor variable
x_smote , y_smote = smote.fit_sample(X_train, Y_train)
x_train1, y_train1 = x_smote , y_smote

In [143]:
x_train1, y_train1=loras.fit_resample()

In [265]:
#scaler = StandardScaler()  

#scaler.fit(x_train1)  
#x_train1 = scaler.transform(x_train1)  

#x_test1 = scaler.transform(X_test)
x_test1 =X_test

#m = MLPClassifier(hidden_layer_sizes=(132,132))
m=LogisticRegression()
m.fit(x_train1, y_train1)
predict_y = m.predict(x_test1)

print('ROCAUC score:',roc_auc_score(Y_test, predict_y))
print('Accuracy score:',accuracy_score(Y_test, predict_y))
print('F1 score:',f1_score(Y_test, predict_y))
print(classification_report(Y_test, predict_y))
confusion_matrix(Y_test, predict_y)

ROCAUC score: 0.7718270427208627
Accuracy score: 0.7935097147581646
F1 score: 0.02346041055718475
              precision    recall  f1-score   support

           0       1.00      0.79      0.88      4822
           1       0.01      0.75      0.02        16

    accuracy                           0.79      4838
   macro avg       0.51      0.77      0.45      4838
weighted avg       1.00      0.79      0.88      4838



C:\Users\zys\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


array([[3827,  995],
       [   4,   12]], dtype=int64)

In [266]:
m=RandomForestClassifier()
m.fit(x_train1, y_train1)
predict_y = m.predict(x_test1)
print('ROCAUC score:',roc_auc_score(Y_test, predict_y))
print('Accuracy score:',accuracy_score(Y_test, predict_y))
print('F1 score:',f1_score(Y_test, predict_y))
print(classification_report(Y_test, predict_y))
confusion_matrix(Y_test, predict_y)

ROCAUC score: 0.71875
Accuracy score: 0.9981397271599834
F1 score: 0.6086956521739131
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      4822
           1       1.00      0.44      0.61        16

    accuracy                           1.00      4838
   macro avg       1.00      0.72      0.80      4838
weighted avg       1.00      1.00      1.00      4838



array([[4822,    0],
       [   9,    7]], dtype=int64)

In [267]:
X_train, X_test, y_train, y_test = train_test_split(dataset_[:,:-1],
                                                    dataset_[:,-1],
                                                    test_size=0.3,
                                                    stratify=dataset_[:,-1]
                                                    )


pipeline = imbpipeline(steps = [['smote', SMOTE()],
                                ['scaler', MinMaxScaler()],
                                ['classifier', RandomForestClassifier()]])

stratified_kfold = StratifiedKFold(n_splits=5,
                                       shuffle=True
                                       )
    
param_grid = {'classifier__n_estimators':[10,20,30,40,50,60,70,80,90,100,150,200,300,400,500]}
grid_search = GridSearchCV(estimator=pipeline,
                           param_grid=param_grid,
                           scoring='f1_macro',
                           cv=stratified_kfold,
                           n_jobs=-1)

grid_search.fit(X_train, y_train)
cv_score_RF_s = grid_search.best_score_
test_score_RF_s = grid_search.score(X_test, y_test)
print(f'Cross-validation score: {cv_score_RF_s}\nTest score: {test_score_RF_s}')

Cross-validation score: 0.689172202682504
Test score: 0.6759011711113683


In [268]:
pipeline = imbpipeline(steps = [['ADASYN', ADASYN()],
                                ['scaler', MinMaxScaler()],
                                ['classifier', RandomForestClassifier()]])

stratified_kfold = StratifiedKFold(n_splits=5,
                                       shuffle=True
                                       )
    
param_grid = {'classifier__n_estimators':[10,20,30,40,50,60,70,80,90,100,150,200,300,400,500]}
grid_search = GridSearchCV(estimator=pipeline,
                           param_grid=param_grid,
                           scoring='f1_macro',
                           cv=stratified_kfold,
                           n_jobs=-1)

grid_search.fit(X_train, y_train)
cv_score_RF_a = grid_search.best_score_
test_score_RF_a = grid_search.score(X_test, y_test)
print(f'Cross-validation score: {cv_score_RF_a}\nTest score: {test_score_RF_a}')

Cross-validation score: 0.6510326187220229
Test score: 0.7217045166931272


In [269]:
pipeline = imbpipeline(steps = [['SVMSMOTE', SVMSMOTE()],
                                ['scaler', MinMaxScaler()],
                                ['classifier', RandomForestClassifier()]])

stratified_kfold = StratifiedKFold(n_splits=5,
                                       shuffle=True
                                       )
    
param_grid = {'classifier__n_estimators':[10,20,30,40,50,60,70,80,90,100,150,200,300,400,500]}
grid_search = GridSearchCV(estimator=pipeline,
                           param_grid=param_grid,
                           scoring='f1_macro',
                           cv=stratified_kfold,
                           n_jobs=-1)

grid_search.fit(X_train, y_train)
cv_score_RF_svm = grid_search.best_score_
test_score_RF_svm = grid_search.score(X_test, y_test)
print(f'Cross-validation score: {cv_score_RF_svm}\nTest score: {test_score_RF_svm}')

Cross-validation score: 0.651032618722023
Test score: 0.7217045166931272


In [270]:
pipeline = imbpipeline(steps = [['SMOTE', SMOTE()],
                                ['scaler', MinMaxScaler()],
                                ['classifier', LogisticRegression()]])
    
param_grid = {'classifier__C':[0.001, 0.01, 0.1, 1, 10, 100, 1000]}
grid_search = GridSearchCV(estimator=pipeline,
                           param_grid=param_grid,
                           scoring='f1_macro',
                           cv=stratified_kfold,
                           n_jobs=-1)

grid_search.fit(X_train, y_train)
cv_score_LR_s = grid_search.best_score_
test_score_LR_s= grid_search.score(X_test, y_test)
print(f'Cross-validation score: {cv_score_LR_s}\nTest score: {test_score_LR_s}')

Cross-validation score: 0.4638708400596904
Test score: 0.4599437600016514


C:\Users\zys\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [271]:
pipeline = imbpipeline(steps = [['ADASYN', ADASYN()],
                                ['scaler', MinMaxScaler()],
                                ['classifier', LogisticRegression()]])
    
param_grid = {'classifier__C':[0.001, 0.01, 0.1, 1, 10, 100, 1000]}
grid_search = GridSearchCV(estimator=pipeline,
                           param_grid=param_grid,
                           scoring='f1_macro',
                           cv=stratified_kfold,
                           n_jobs=-1)

grid_search.fit(X_train, y_train)
cv_score_LR_a = grid_search.best_score_
test_score_LR_a= grid_search.score(X_test, y_test)
print(f'Cross-validation score: {cv_score_LR_a}\nTest score: {test_score_LR_a}')

Cross-validation score: 0.4641704005633172
Test score: 0.45767509400495365


C:\Users\zys\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [272]:
pipeline = imbpipeline(steps = [['SVMSMOTE', SVMSMOTE()],
                                ['scaler', MinMaxScaler()],
                                ['classifier', LogisticRegression()]])
    
param_grid = {'classifier__C':[0.001, 0.01, 0.1, 1, 10, 100, 1000]}
grid_search = GridSearchCV(estimator=pipeline,
                           param_grid=param_grid,
                           scoring='f1_macro',
                           cv=stratified_kfold,
                           n_jobs=-1)

grid_search.fit(X_train, y_train)
cv_score_LR_svm = grid_search.best_score_
test_score_LR_svm= grid_search.score(X_test, y_test)
print(f'Cross-validation score: {cv_score_LR_svm}\nTest score: {test_score_LR_svm}')

Cross-validation score: 0.5229216606259434
Test score: 0.5631164400742488


In [273]:
pipeline = imbpipeline(steps = [['SMOTE', SMOTE()],
                                ['scaler', MinMaxScaler()],
                                ['classifier', DecisionTreeClassifier()]])
    
param_grid = {'classifier__criterion':['gini','entropy'],'classifier__max_depth':[4,5,6,7,8,9,10,11,12,15,20,30,40,50,70,90,120,150,200,300,400,500]}
grid_search = GridSearchCV(estimator=pipeline,
                           param_grid=param_grid,
                           scoring='f1_macro',
                           cv=stratified_kfold,
                           n_jobs=-1)

grid_search.fit(X_train, y_train)
cv_score_DT_s = grid_search.best_score_
test_score_DT_s= grid_search.score(X_test, y_test)
print(f'Cross-validation score: {cv_score_DT_s}\nTest score: {test_score_DT_s}')

Cross-validation score: 0.6279275548703435
Test score: 0.5912065839567957


In [274]:
pipeline = imbpipeline(steps = [['ADASYN', ADASYN()],
                                ['scaler', MinMaxScaler()],
                                ['classifier', DecisionTreeClassifier()]])
    
param_grid = {'classifier__criterion':['gini','entropy'],'classifier__max_depth':[4,5,6,7,8,9,10,11,12,15,20,30,40,50,70,90,120,150,200,300,400,500]}
grid_search = GridSearchCV(estimator=pipeline,
                           param_grid=param_grid,
                           scoring='f1_macro',
                           cv=stratified_kfold,
                           n_jobs=-1)

grid_search.fit(X_train, y_train)
cv_score_DT_a = grid_search.best_score_
test_score_DT_a= grid_search.score(X_test, y_test)
print(f'Cross-validation score: {cv_score_DT_a}\nTest score: {test_score_DT_a}')

Cross-validation score: 0.6260475512366629
Test score: 0.6010569263091474


In [275]:
pipeline = imbpipeline(steps = [['SVMSMOTE',SVMSMOTE()],
                                ['scaler', MinMaxScaler()],
                                ['classifier', DecisionTreeClassifier()]])
    
param_grid = {'classifier__criterion':['gini','entropy'],'classifier__max_depth':[4,5,6,7,8,9,10,11,12,15,20,30,40,50,70,90,120,150,200,300,400,500]}
grid_search = GridSearchCV(estimator=pipeline,
                           param_grid=param_grid,
                           scoring='f1_macro',
                           cv=stratified_kfold,
                           n_jobs=-1)

grid_search.fit(X_train, y_train)
cv_score_DT_svm = grid_search.best_score_
test_score_DT_svm= grid_search.score(X_test, y_test)
print(f'Cross-validation score: {cv_score_DT_svm}\nTest score: {test_score_DT_svm}')

Cross-validation score: 0.6257767149497375
Test score: 0.6096588289534348


In [278]:
pipeline = imbpipeline(steps = [['SMOTE', SMOTE()],
                                ['scaler', MinMaxScaler()],
                                ['classifier', KNeighborsClassifier()]])
    
param_grid = {'classifier__n_neighbors':[1,2,3,4,5,6,7,8,9,10,20,30,40,50,80,100,200,300,400,500,600,700,800,900,1000]}
grid_search = GridSearchCV(estimator=pipeline,
                           param_grid=param_grid,
                           scoring='f1_macro',
                           cv=stratified_kfold,
                           n_jobs=-1)

grid_search.fit(X_train, y_train)
cv_score_knn_s = grid_search.best_score_
test_score_knn_s= grid_search.score(X_test, y_test)
print(f'Cross-validation score: {cv_score_knn_s}\nTest score: {test_score_knn_s}')

Cross-validation score: 0.5261014139250931
Test score: 0.6156210847975554


In [279]:
pipeline = imbpipeline(steps = [['ADASYN', ADASYN()],
                                ['scaler', MinMaxScaler()],
                                ['classifier', KNeighborsClassifier()]])
    
param_grid = {'classifier__n_neighbors':[1,2,3,4,5,6,7,8,9,10,20,30,40,50,80,100,200,300,400,500,600,700,800,900,1000]}
grid_search = GridSearchCV(estimator=pipeline,
                           param_grid=param_grid,
                           scoring='f1_macro',
                           cv=stratified_kfold,
                           n_jobs=-1)

grid_search.fit(X_train, y_train)
cv_score_knn_a = grid_search.best_score_
test_score_knn_a= grid_search.score(X_test, y_test)
print(f'Cross-validation score: {cv_score_knn_a}\nTest score: {test_score_knn_a}')

Cross-validation score: 0.5400147539820169
Test score: 0.6180261790982755


In [280]:
pipeline = imbpipeline(steps = [['SVMSMOTE', SVMSMOTE()],
                                ['scaler', MinMaxScaler()],
                                ['classifier', KNeighborsClassifier()]])
    
param_grid = {'classifier__n_neighbors':[1,2,3,4,5,6,7,8,9,10,20,30,40,50,80,100,200,300,400,500,600,700,800,900,1000]}
grid_search = GridSearchCV(estimator=pipeline,
                           param_grid=param_grid,
                           scoring='f1_macro',
                           cv=stratified_kfold,
                           n_jobs=-1)

grid_search.fit(X_train, y_train)
cv_score_knn_svm = grid_search.best_score_
test_score_knn_svm= grid_search.score(X_test, y_test)
print(f'Cross-validation score: {cv_score_knn_svm}\nTest score: {test_score_knn_svm}')

Cross-validation score: 0.5913627675686772
Test score: 0.7214449396022616


In [285]:
pipeline = imbpipeline(steps = [['SMOTE', SMOTE()],
                                ['scaler', MinMaxScaler()],
                                ['classifier', XGBClassifier()]])
    
param_grid = {'classifier__min_child_weight': [1, 5, 10],
        'classifier__max_depth': [3, 4, 5]}
grid_search = GridSearchCV(estimator=pipeline,
                           param_grid=param_grid,
                           scoring='f1_macro',
                           cv=stratified_kfold,
                           n_jobs=-1)

grid_search.fit(X_train, y_train)
cv_score_xgb_s = grid_search.best_score_
test_score_xgb_s= grid_search.score(X_test, y_test)
print(f'Cross-validation score: {cv_score_xgb_s}\nTest score: {test_score_xgb_s}')

C:\Users\zys\Anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[06:25:16] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Cross-validation score: 0.6828230356731648
Test score: 0.6993786246893123


In [288]:
pipeline = imbpipeline(steps = [['ADASYN', ADASYN()],
                                ['scaler', MinMaxScaler()],
                                ['classifier', XGBClassifier()]])

grid_search = GridSearchCV(estimator=pipeline,
                           param_grid=param_grid,
                           scoring='f1_macro',
                           cv=stratified_kfold,
                           n_jobs=-1)

grid_search.fit(X_train, y_train)
cv_score_xgb_a = grid_search.best_score_
test_score_xgb_a= grid_search.score(X_test, y_test)
print(f'Cross-validation score: {cv_score_xgb_a}\nTest score: {test_score_xgb_a}')

C:\Users\zys\Anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[06:27:31] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Cross-validation score: 0.7010769449107859
Test score: 0.7602997941617609


In [289]:
pipeline = imbpipeline(steps = [['SVMSMOTE', SVMSMOTE()],
                                ['scaler', MinMaxScaler()],
                                ['classifier', XGBClassifier()]])
    
grid_search = GridSearchCV(estimator=pipeline,
                           param_grid=param_grid,
                           scoring='f1_macro',
                           cv=stratified_kfold,
                           n_jobs=-1)

grid_search.fit(X_train, y_train)
cv_score_xgb_svm = grid_search.best_score_
test_score_xgb_svm= grid_search.score(X_test, y_test)
print(f'Cross-validation score: {cv_score_xgb_svm}\nTest score: {test_score_xgb_svm}')

C:\Users\zys\Anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[06:28:13] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Cross-validation score: 0.660159602849007
Test score: 0.7217045166931272


In [290]:
models = pd.DataFrame({
    'Model': ['Random Forest', 'Logistic Regression', 
              'Decision Tree','KNN',  
              'XGBoost'],
    'SMOTE': [cv_score_RF_s,cv_score_LR_s,cv_score_DT_s,cv_score_knn_s,cv_score_xgb_s],
    'ADASYN': [cv_score_RF_a,cv_score_LR_a,cv_score_DT_a,cv_score_knn_a,cv_score_xgb_a],
    'SVMSMOTE':[cv_score_RF_svm,cv_score_LR_svm,cv_score_DT_svm,cv_score_knn_svm,cv_score_xgb_svm]})

In [291]:
models

,Model,SMOTE,ADASYN,SVMSMOTE
0,Random Forest,0.689172,0.651033,0.651033
1,Logistic Regression,0.463871,0.464170,0.522922
2,Decision Tree,0.627928,0.626048,0.625777
3,KNN,0.526101,0.540015,0.591363
4,XGBoost,0.682823,0.701077,0.660160


# To do:
1. add features
2. more algorithms
3. bayesian optimisation
4. oversampling strategy?